In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')

In [ ]:
train= train.drop('id', axis=1)

In [ ]:
test = test.drop('id',axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder= LabelEncoder().fit(train['target'])

In [ ]:
encoder

In [ ]:
train['target'] = encoder.transform(train['target'])

In [ ]:
train['target'].unique()

In [ ]:
x = train.drop(['target'],axis=1)

In [ ]:
y = train['target']

In [ ]:
x.head(3)

In [ ]:
y.head(3)

In [ ]:
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [ ]:
# A parameter grid for XGBoost
params2 = {
        'min_child_weight': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'gamma': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.8,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0,2.1,2.2,2.3,2.4,2,5],
        'subsample': [0.5,0.6,0.7,0.8,0.9,1.0],
        'colsample_bytree': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],
        'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        }

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1997)

In [ ]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

In [ ]:
model = XGBClassifier(learning_rate=0.02,
                      num_class=4, 
                      objective='multi:softprob',
                      tree_method='gpu_hist',
                      n_estimators=600,
                      #predictor='gpu_predictor',
                      silent=True)

In [ ]:
param_comb=5
for folds in range(2,6):
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1997) 
    grid = GridSearchCV(estimator=model, param_grid=params2, n_jobs=16, cv=skf.split(x,y), verbose=3 )
    grid.fit(x, y)
    #print('\n All results:')
    #print(grid.cv_results_)
    print('\n Grid Best estimator:')
    print(grid.best_estimator_)
    print('\n Grid Best score:')
    print(grid.best_score_ * 2 - 1)
    print('\n Grid Best parameters:')
    print(grid.best_params_)
    gresults = pd.DataFrame(grid.cv_results_)
    gresults.to_csv('Grid-search-folds{}.csv'.format(folds), index=False)
    
    test_y_grid = grid.predict_proba(test)
    index = submission.columns.values.tolist()
    grid_submission = pd.DataFrame(test_y_grid,columns = index[1:5])
    grid_submission['id'] = submission['id']
    submission2 = grid_submission[['id','Class_1','Class_2','Class_3','Class_4']]
    submission2.to_csv('Gsubmission{}.csv'.foramt(folds),index=False)